<a href="https://colab.research.google.com/github/tracydo99/bus118/blob/main/Group_Project_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-community langchain-core langchain-openai
!pip install faiss-cpu langchain-text-splitters langgraph
!pip install langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.2
    Uninstalling langchain-core-1.1.2:
      Successfully uninstalled langchain-core-1.1.2
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.2
    Uninstalling lang

In [2]:
!pip install -U pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.4
    Uninstalling pydantic_core-2.41.4:
      Successfully uninstalled pydantic_core-2.41.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [3]:
from typing import TypedDict, List, Annotated, Literal
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage

In [4]:
import os
os.environ['OPENAI_API_KEY'] = '#####'
print(f"API Key successfully set (showing first 5 chars): {os.environ.get('OPENAI_API_KEY')[:5]}...")

API Key successfully set (showing first 5 chars): sk-pr...


In [5]:
# --- Define State and LLM ---
class ITSupportState(TypedDict):
    messages: Annotated[List[HumanMessage], add_messages]
    classification: str
    retrieved_docs: List[str]
    automation_success: bool
    automation_output: str
    draft_response: str
    needs_escalation: bool

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
loader = TextLoader("/content/it_kb.txt")
documents = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("Data successfully embedded and indexed in the FAISS in-memory vector store.")

def retrieve_knowledge(query: str) -> List[str]:
    """Retrieves relevant text snippets from the FAISS vector store."""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    retrieved_docs = retriever.invoke(query)
    return [doc.page_content for doc in retrieved_docs]

Data successfully embedded and indexed in the FAISS in-memory vector store.


In [8]:
@tool
def reset_password(employee_id: str) -> str:
    """Automates the password reset process for an employee using their ID."""
    if len(employee_id) == 6 and employee_id.isdigit():
        temp_password = "Tsla" + employee_id[-2:] + "$25"
        return f"SUCCESS: Password reset initiated for EID {employee_id}. Temporary password issued to secondary email: {temp_password}"
    else:
        return "FAILURE: User ID format invalid or unknown error. Unable to perform automated reset."

@tool
def create_jira_ticket(summary: str, severity: Literal["Severity 1 (Critical)", "Severity 2 (High)", "Severity 3 (Medium)"]) -> str:
    """Creates a new IT support ticket in the Jira system for human intervention."""
    ticket_id = f"TSLA-IT-{hash(summary) % 10000}"
    return f"TICKET_CREATED: {ticket_id}. Summary: '{summary}'. Severity: {severity}. Case successfully escalated to the human IT team."

automation_tools = [reset_password, create_jira_ticket]

In [9]:
def intake_agent(state: ITSupportState) -> dict:
    """Deterministically classifies the request and routes to the next agent."""
    import re

    user_input = state["messages"][-1].content
    text = user_input.lower()

    # 1) AUTOMATION: password reset / unlock + 6-digit employee ID
    has_password = "password" in text or "account lock" in text or "unlock" in text
    has_eid = re.search(r"\b\d{6}\b", text) is not None

    if has_password and has_eid:
        classification = "AUTOMATION"

    # 2) KNOWLEDGE: policy / process / how-to questions
    elif any(
        kw in text
        for kw in [
            "policy",
            "process",
            "how do i",
            "how to",
            "creating new employee accounts",
            "new employee account",
            "error",
            "update",
            "software update",
            "network",
            "wifi",
            "wi-fi",
            "internet",
            "connection issue",
            "troubleshooting",
        ]
    ):
        classification = "KNOWLEDGE"

    # 3) ESCALATE
    else:
        classification = "ESCALATE"

    return {
        "classification": classification,
        "messages": state["messages"]
        + [HumanMessage(content=f"Intake: Classified as {classification}.")],
        "draft_response": None,
    }

def knowledge_agent(state: ITSupportState) -> dict:
    """Retrieves context via RAG and synthesizes an answer using LLM."""
    original_user_query = state['messages'][0].content
    retrieved_docs = retrieve_knowledge(original_user_query)

    synthesis_prompt = (
        "You are the Knowledge Agent for Tesla IT Support. Your sole purpose is to provide a precise and concise answer to the user's query. "
        "You **must only** use the 'Retrieved Context' provided below. Do not use any prior knowledge. "
        "If the provided 'Retrieved Context' does not contain enough information to directly answer the user's question, "
        "you must politely state that you cannot answer based on the available information. "
        "Do not attempt to guess or infer information not explicitly present in the context."+
        f"\n\nRetrieved Context:\n{'\n-----'.join(retrieved_docs)}"
    )
    prompt = ChatPromptTemplate.from_messages([("system", synthesis_prompt), ("human", "{input}")])

    draft_response = llm.invoke(prompt.invoke({"input": original_user_query})).content

    return {"retrieved_docs": retrieved_docs,
            "draft_response": draft_response,
            "messages": state['messages'] + [HumanMessage(content="Knowledge: Retrieval and synthesis complete.")]}

def workflow_agent(state: ITSupportState) -> dict:
    """Handles automation tasks like password resets."""
    system_prompt = (
        "You are the Workflow Automation Bot for Tesla IT Support. "
        "The user's request has been classified as AUTOMATION, and you are to perform a password reset. "
        "Your primary task is to extract the **employee ID (EID)** from the user's message "
        "and then call the `reset_password` tool with the extracted EID. "
        "It is critical that you invoke the `reset_password` tool."
    )

    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{input}")])
    agent_chain = prompt | llm.bind_tools(automation_tools, tool_choice="reset_password")

    try:
        ai_message = agent_chain.invoke({"input": state['messages'][0].content})

        output = "FAILURE: LLM did not call a tool for automation."
        success = False

        state['messages'].append(ai_message)

        if ai_message.tool_calls:
            tool_call = ai_message.tool_calls[0]

            if tool_call['name'] == 'reset_password':
                employee_id = tool_call['args'].get('employee_id', 'N/A')

                output = reset_password.func(employee_id)

                success = output.startswith("SUCCESS")
            else:
                output = f"FAILURE: LLM called unexpected tool: {tool_call['name']} in workflow_agent."
        else:
            output = "FAILURE: LLM failed to call the specified 'reset_password' tool."

        return {
            "automation_success": success,
            "automation_output": output,
            "messages": state['messages'] + [HumanMessage(content=f"Workflow: {output}")]
        }

    except Exception as e:
        return {
            "automation_success": False,
            "automation_output": f"CRITICAL FAILURE: LCEL Chain execution error: {str(e)}",
            "messages": state['messages'] + [HumanMessage(content=f"Workflow: Execution Error: {str(e)}")]
        }


def escalation_agent(state: ITSupportState) -> dict:
    """Creates a Jira ticket using the tool for human handover."""
    original_query = state['messages'][0].content
    automation_outcome = state.get('automation_output', 'Automation not attempted or failed.')
    failure_summary = f"Automation failed. Classification: {state['classification']}. Tool Output: {automation_outcome}"

    ticket_output = create_jira_ticket.func(failure_summary, "Severity 2 (High)")

    return {"draft_response": ticket_output,
            "messages": state['messages'] + [HumanMessage(content=f"Escalation: {ticket_output}")]}

def router_intake(state: ITSupportState) -> str:
    """Determines the next step based on the Intake Agent's classification."""
    if state['classification'] == "AUTOMATION":
        return "workflow_agent"
    elif state['classification'] == "KNOWLEDGE":
        return "knowledge_agent"
    else: # ESCALATE
        return "escalation_agent"

def draft_final_response(state: ITSupportState) -> dict:
    """Generates the final user-facing response."""
    response = state.get('automation_output') or state.get('draft_response') or "I'm not sure how to resolve this. I am routing your request to a human."

    return {"draft_response": response,
            "messages": state['messages'] + [HumanMessage(content=f"Final Answer: {response}")]}

In [10]:
graph_builder = StateGraph(ITSupportState)

graph_builder.add_node("intake_agent", intake_agent)
graph_builder.add_node("knowledge_agent", knowledge_agent)
graph_builder.add_node("workflow_agent", workflow_agent)
graph_builder.add_node("escalation_agent", escalation_agent)
graph_builder.add_node("draft_final_response", draft_final_response)

graph_builder.set_entry_point("intake_agent")

graph_builder.add_conditional_edges(
    "intake_agent",
    router_intake,
    {
        "workflow_agent": "workflow_agent",
        "knowledge_agent": "knowledge_agent",
        "escalation_agent": "escalation_agent",
    }
)

graph_builder.add_edge("knowledge_agent", "draft_final_response")

graph_builder.add_conditional_edges(
    "workflow_agent",
    lambda x: "success" if x["automation_success"] else "fail",
    {"success": "draft_final_response", "fail": "escalation_agent"}
)

graph_builder.add_edge("escalation_agent", END)
graph_builder.add_edge("draft_final_response", END)

app = graph_builder.compile()

In [11]:
print("\n--- Running Test Scenario 1: Successful Automation (Password Reset) ---")
test_query_success = "I forgot my password. My employee ID is 987654. Please reset it."
final_state_success = app.invoke({"messages": [HumanMessage(content=test_query_success)]})
print(f"\nFinal State Response (Success):\n{final_state_success['draft_response']}")

print("\n--- Running Test Scenario 2: RAG Retrieval (Policy Query) ---")
test_query_rag = "What is the policy for creating new employee accounts?"
final_state_rag = app.invoke({"messages": [HumanMessage(content=test_query_rag)]})
print(f"\nFinal State Response (RAG):\n{final_state_rag['draft_response']}")


--- Running Test Scenario 1: Successful Automation (Password Reset) ---

Final State Response (Success):
SUCCESS: Password reset initiated for EID 987654. Temporary password issued to secondary email: Tsla54$25

--- Running Test Scenario 2: RAG Retrieval (Policy Query) ---

Final State Response (RAG):
The policy for creating new employee accounts is as follows:

1. HR must submit and approve the new hire request before account creation.
2. The username format is first initial + last name (Example: JSmith).
3. Default access groups include:
   - Global-All-Employees
   - Intranet-Access
   - TSLA-Basic-VPN
4. The account must be provisioned within 4 hours after HR approval.
5. If account creation takes longer than 4 hours, escalate as Severity 1 to the Identity Management Team.


In [ ]:
print("\n\n=======================================================")
print("🚀 IT SUPPORT AI: START CHAT (Type 'quit' or 'exit' to end)")
print("=======================================================")

while True:
    user_input = input("You: ")

    if user_input.lower() in ['quit', 'exit']:
        print("IT Support AI: Goodbye!")
        break

    input_message = {"messages": [HumanMessage(content=user_input)]}

    try:
        final_state = app.invoke(input_message)

        ai_response = final_state.get('draft_response', "Sorry, I couldn't process that request.")

        print(f"AI: {ai_response}\n")

    except Exception as e:
        print(f"AI: A critical error occurred during processing: {e}")
        print("AI: Routing to human support now.")
        break



🚀 IT SUPPORT AI: START CHAT (Type 'quit' or 'exit' to end)
